In [181]:
# go into requirements.txt 
# !pip -q install pandas
# !pip -q install nltk
# !pip -q install torchtext
# !pip install -q torch 
# !pip install -q pytorch_lightning
# !pip install -q torchvision
# !pip -q install torch
#!pip -q install sklearn

In [182]:
import pandas as pd
import random 
import zipfile
import numpy as np
import torch 
# with zipfile.ZipFile('./data/ag-news-classification-dataset.zip') as zf:
#     zf.extractall('./data/')

In [183]:
# datasets.py
trainpath = '../data/train.csv'
testpath = '../data/test.csv'
train = pd.read_csv(trainpath)
test = pd.read_csv(testpath)
train.head(5)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [184]:
# concatenate Title and Description
# clean and tokenize 
from torchtext.data import get_tokenizer
from nltk.corpus import stopwords
import nltk
import re
try:
    stopwords = set(stopwords.words('english'))
except LookupError as e:
    print(f"{e}, {e.__class__}")
    nltk.download('stopwords')
    stopwords = set(stopwords.words('english'))
    

In [185]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/P76114511/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [186]:
def preproc(title, descrip):
    text = title + '.' + descrip
    text = re.sub(r'[~`\-!@#$%^&*():;"{}_/?><\|.,`0-9\\]',' ',text)
    # text = nltk.word_tokenize(text)     # as suggested by senpai 
    text = text.split()
    text = [tok.lower() for tok in text if tok not in stopwords]
    return text
from torchtext.vocab import build_vocab_from_iterator
def build_vocab(datasets):
      for dataset in datasets:
        for rid, data in dataset.iterrows():
            text = data['preproc_text']
            yield text
tokenizer = get_tokenizer("basic_english")
train['preproc_text'] = train.apply(lambda x:preproc(x['Title'], x['Description']), axis=1)
test['preproc_text'] = test.apply(lambda x:preproc(x['Title'], x['Description']), axis=1)
train['label'] = train['Class Index'].apply(lambda x:x-1)
test['label'] = test['Class Index'].apply(lambda x:x-1)
# train['Class Index'].unique() # array([3, 4, 2, 1])

# class IterableTexts:
#   def __iter__(self):
#     for dataset in [trainset, testset]:
#         for data in dataset:
#             text = data['text']
#             yield list(x.lower() for x in tokenizer(text) if x.lower() not in stopwords)

In [187]:
train['label'].value_counts()

2    30000
3    30000
1    30000
0    30000
Name: label, dtype: int64

tensor([2, 2, 2,  ..., 1, 1, 1])

In [188]:
vocabs = build_vocab_from_iterator(build_vocab([train, test]), 
                                   specials=["[PAD]", "[UNK]"], 
                                   min_freq = Config['MIN_FREQ'],
                                   max_tokens =None)
vocabs.set_default_index(vocabs["[UNK]"]) # let padding be index 0 
print(len(vocabs))

28372


## Config

In [189]:
Config = {
    'IS_BIDIRECTIONAL':False,
    'SEED':42,
    'NUM_WORKER':12,
    'BATCHSIZE': 200,
    'LR':5e-4,
    'NUM_LAYERS':2,
    'MAXLEN':50,
    'MIN_FREQ':5,
    'DATASET':'AGNews', 
    'EPOCHS':20,
    'WEIGHT_DECAY':1e-5,  #default:0
    'DROPOUT':0.3, 
    'CLIP_GRAD':0.5,
    'NUMCHOICE':4,
    'LSTM_HDIM':256, 
    'EMBDIM':32,
    'HIDDIM':32
}
LabelMapping = {0:"World",
                1:"Sports",
                2:"Business",
                3:"Sci/Tech"}

In [213]:
def tokenize(x):
    maxlen = Config['MAXLEN']            # nltk.word_tokenize, spacy (better tokenize)
    x = x[:maxlen]                      # truncation
    tokenized_x = np.zeros(maxlen)      # padding: np.ones since [PAD]: 0
    tokenized_x[:len(x)] = np.array([vocabs[v] for v in x])
    return tokenized_x
trainX = train['preproc_text'].apply(lambda x:tokenize(x))
trainy = train['label']
from collections import Counter
Counter(trainy)

Counter({2: 30000, 3: 30000, 1: 30000, 0: 30000})

In [191]:
trainX = torch.tensor(np.vstack(trainX), dtype=torch.long)
trainy = torch.tensor(trainy, dtype=torch.long)

In [192]:
testX = test['preproc_text'].apply(lambda x:tokenize(x))
testy = test['label']
testX = torch.tensor(np.vstack(testX), dtype=torch.long)
testy = torch.tensor(testy, dtype=torch.long)

In [193]:
print(trainX.shape)
print(trainy.shape)
print(testX.shape)
print(testy.shape)

torch.Size([120000, 50])
torch.Size([120000])
torch.Size([7600, 50])
torch.Size([7600])


In [194]:
# wrap into dataloaders, collator and shuffling 
# Dataloader 
from torch.utils.data import TensorDataset, DataLoader 
trainset = TensorDataset(trainX, trainy)
testset = TensorDataset(testX, testy)
trainloader = DataLoader(trainset, shuffle = True,
                         num_workers = Config['NUM_WORKER'],
                         batch_size=Config['BATCHSIZE'])
testloader = DataLoader(testset, shuffle = False, 
                        num_workers = Config['NUM_WORKER'],
                        batch_size=Config['BATCHSIZE'])

In [195]:
r = random.randint(0, trainX.shape[0])
detokenizer = vocabs.get_itos()
print(f'Vocab Size(including specials): {len(vocabs.get_itos())}')
print(f'random index: {r}')

# “World”: 0,“Sports”: 1,“Business”: 2,“Sci/Tech”: 3

x = trainX[r,:]
print(f'tokenized example:\n {x}')
print(f'label: {LabelMapping[trainy[r].item()]}')
print(f'detokenized example:')
print([detokenizer[tok] for tok in x])

Vocab Size(including specials): 28372
random index: 3278
tokenized example:
 tensor([ 8722,  1814,   154,   712,  4938,    11,    48,   271,   154,  1523,
         2725,   321,  1018,   418,  8722,  3547,  5455,  5296,  4605,  2116,
            2,  1018,   852,   336, 15302,     1,    28,    38,   334,     4,
           13,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
label: Sci/Tech
detokenized example:
['compuware', 'blasts', 'ibm', 'legal', 'tactics', 'two', 'years', 'ago', 'ibm', 'ordered', 'produce', 'source', 'code', 'products', 'compuware', 'identified', 'containing', 'pirated', 'intellectual', 'property', 'the', 'code', 'missing', 'but', 'lo', '[UNK]', 'last', 'week', 'called', 'said', 'quot', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [196]:
Config['VOCAB_SIZE'] = len(vocabs)

## Model Architecture

In [197]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import torchmetrics # Accuracy
pl.utilities.seed.seed_everything(Config['SEED'])

# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Embedding, Dropout

# model = Sequential()
# model.add(Embedding(input_dim= Config['VOCAB_SIZE'], 
#                     output_dim= Config['EMBDIM'],  
#                     input_length=Config['MAXLEN'], name = 'embedding'))
# model.add(LSTM(Config['LSTM_HDIM'],name = 'lstm')) 
# model.add(Dense(Config['HIDDIM'], name = 'fc')) # 這裡不應該有softmax
# model.add(Dropout(0.3, name = 'dropout'))
# model.add(Dense(Config['NUMCHOICE'], activation="softmax", name = 'classifier'))
# model.summary()

class LSTMNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # variables
        self.num_directions = 2 if Config['IS_BIDIRECTIONAL'] else 1
        self.LossFn = nn.CrossEntropyLoss(
            ignore_index=vocabs['PAD']
        )
        self.lstm_hdim = Config['LSTM_HDIM']
        
        # layers 
        self.Embedding = nn.Embedding(Config['VOCAB_SIZE'], Config['EMBDIM'])
        # https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
        self.Embedding.weight.data.uniform_(-1, 1)

        self.LSTM = nn.LSTM(
            input_size = Config['EMBDIM'],
            hidden_size = Config['LSTM_HDIM'],
            num_layers = Config['NUM_LAYERS'],
            bidirectional = Config['IS_BIDIRECTIONAL'],
            batch_first = True,
        )
        self.FC = nn.Linear(Config['LSTM_HDIM']*self.num_directions, 
                            Config['HIDDIM'])
        self.Dropout = nn.Dropout(Config['DROPOUT'])  
        self.Out  = nn.Linear(Config['HIDDIM'], Config['NUMCHOICE'])
        
        # metrics
        self.accuracy = torchmetrics.Accuracy()
    
    def forward(self, x):
        # https://zhenglungwu.medium.com/pytorch%E5%AF%A6%E4%BD%9Clstm%E5%9F%B7%E8%A1%8C%E8%A8%8A%E8%99%9F%E9%A0%90%E6%B8%AC-d1d3f17549e7
        x = self.Embedding(x)
        x, (h, c) = self.LSTM(x)
        x = self.FC(x)
        x = self.Dropout(x)
        x = self.Out(x)
        x = x[:, -1]
        return x
    
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        X, labels = batch 
        preds = self(X)
        loss = self.LossFn(preds, labels)
        accuracy = self.accuracy(torch.argmax(preds, axis=1), 
                                 labels)
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_acc', accuracy, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        X, labels = batch 
        preds = self(X)
        loss = self.LossFn(preds, labels)
        accuracy = self.accuracy(torch.argmax(preds, axis=1), labels)
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_acc', accuracy, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), 
                                     lr=Config['LR'])
        return optimizer

Global seed set to 42


In [198]:
import torch
from torchmetrics import Accuracy
target = torch.tensor([0, 3])
preds = torch.tensor([[0, 0.5, 0.8, 0.2],
                      [0.2,0.1,0.1,0.3]])
preds = torch.argmax(preds, axis=1)
print(preds)
accuracy = Accuracy()
accuracy(preds, target)

tensor([2, 3])


tensor(0.5000)

In [199]:
# set up Lightning model architecture 
model = LSTMNet()
print(model)

LSTMNet(
  (LossFn): CrossEntropyLoss()
  (Embedding): Embedding(28372, 32)
  (LSTM): LSTM(32, 256, num_layers=2, batch_first=True)
  (FC): Linear(in_features=256, out_features=32, bias=True)
  (Dropout): Dropout(p=0.3, inplace=False)
  (Out): Linear(in_features=32, out_features=4, bias=True)
  (accuracy): Accuracy()
)


In [200]:
# Trainer.fit 

pl.utilities.seed.seed_everything(Config['SEED'])
trainer = pl.Trainer(max_epochs = Config['EPOCHS'], 
                     gradient_clip_val=Config['CLIP_GRAD'],
                     accelerator="gpu", 
                     devices=1)
trainer.fit(model, trainloader, testloader)

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | LossFn    | CrossEntropyLoss | 0     
1 | Embedding | Embedding        | 907 K 
2 | LSTM      | LSTM             | 823 K 
3 | FC        | Linear           | 8.2 K 
4 | Dropout   | Dropout          | 0     
5 | Out       | Linear           | 132   
6 | accuracy  | Accuracy         | 0     
-----------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.958     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/P76114511/main/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [201]:
trainer.validate(model, dataloaders=testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.6702631711959839
        val_loss            0.4220391511917114
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.4220391511917114, 'val_acc': 0.6702631711959839}]

## Check

In [206]:
preds = []
golds = []
model.eval()

for X, y in testloader:
    with torch.no_grad():
        y_hat = model(X)
        y_hat = torch.argmax(y_hat, axis = 1)
    golds.extend(y.numpy())
    preds.extend(y_hat.numpy())

### Confusion Matrix

In [209]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(golds, preds)
#???? 完全沒有預測 Sports 
cm

array([[1715,    0,   93,   92],
       [1117,    0,  404,  379],
       [  68,    0, 1677,  155],
       [  46,    0,  152, 1702]])

In [208]:
from sklearn.metrics import accuracy_score as SKACC
SKACC(golds, preds)

0.6702631578947369